In [30]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

import numpy as np
import pickle as pk
import os
import pandas as pd
from scipy.spatial.distance import cosine
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
import statsmodels.formula.api as sm
import statsmodels.stats.sandwich_covariance as sw
import statsmodels as statsmodels
from statsmodels.discrete.discrete_model import Logit

# loading metadata

In [31]:
outcome_data = pd.read_stata('/data/Projects/ornelie_dlchen/HundredPercentData_Bloomberg/DTAs/TOUSE/BloombergCASELEVEL_Touse.dta')

In [32]:
outcome_data = outcome_data[['caseid', 'Circuit', 'citation','dissentdummy','reversedummy', 'reverseandremand', 
                             'reversenoremand', 'affirmdummy','Affirmed', 'AffirmedInPart', 'Reversed', 
                             'ReversedInPart', 'Vacated', 'VacatedInPart','Remanded', 'Treat', 'Dissenting1', 
                             'Dissenting2',]]

In [33]:
outcome_data = outcome_data.drop_duplicates("caseid")

# import dataframes

# difference between average opinion in its year and topic and this opinion

In [34]:
opinion_YB_dis = pd.read_csv('/home/jcai/geometry_of_law/data_and_dictionary/opinion_YB_dis.csv')

In [35]:
opinion_YB_dis_merged = opinion_YB_dis.copy().merge(outcome_data, how = "left",on = "caseid")

## the difference between his/her (judge) average opinion demeaned by year and topic and the opinion demeaned by year and topic

In [36]:
opinion_YB_judge_YB_dis = pd.read_csv("/home/jcai/geometry_of_law/data_and_dictionary/opinion_YB_judge_YB_dis.csv")

In [37]:
opinion_YB_judge_YB_dis_merged = opinion_YB_judge_YB_dis.copy().merge(outcome_data, how = "left",on = "caseid")

In [73]:
opinion_YB_judge_YB_dis_merged_case_level = outcome_data.copy().merge(opinion_YB_judge_YB_dis, how = "left",on = "caseid")

# regression models

# Linear Regression With Clustered Error to affirmed

In [38]:
opinion_YB_dis_merged_affirmed = opinion_YB_dis_merged.copy()
opinion_YB_dis_merged_affirmed = opinion_YB_dis_merged_affirmed[opinion_YB_dis_merged_affirmed.affirmdummy.notna()]
opinion_YB_dis_merged_affirmed = opinion_YB_dis_merged_affirmed[opinion_YB_dis_merged_affirmed.dis.notna()]

In [40]:
opinion_YB_judge_YB_dis_merged_affirmed = opinion_YB_judge_YB_dis_merged.copy()
opinion_YB_judge_YB_dis_merged_affirmed = opinion_YB_judge_YB_dis_merged_affirmed[opinion_YB_judge_YB_dis_merged_affirmed.affirmdummy.notna()]
opinion_YB_judge_YB_dis_merged_affirmed = opinion_YB_judge_YB_dis_merged_affirmed[opinion_YB_judge_YB_dis_merged_affirmed.dis.notna()]

# within judge difference

In [41]:
opinion_YB_judge_YB_dis_merged_affirmed.groupby("affirmdummy").count()

,Unnamed: 0,docname,dis,judge_name,party,year,circuit,circuit-year,big-issue,detailed-issue,...,Affirmed,AffirmedInPart,Reversed,ReversedInPart,Vacated,VacatedInPart,Remanded,Treat,Dissenting1,Dissenting2
affirmdummy,,,,,,,,,,,,,,,,,,,,,
0.0,131523,131523,131523,131523,131522,131523,131523,131523,131523,129482,...,131523,131523,131523,131523,131523,131523,131523,120499,15160,121
1.0,157226,157226,157226,157226,157226,157226,157226,157226,157226,154440,...,157226,157226,157226,157226,157226,157226,157226,157226,11353,35


In [42]:
cluster_court_opinion_YB_judge_YB_dis_merged_affirmed_ols = sm.ols(formula='affirmdummy ~ dis', data=opinion_YB_judge_YB_dis_merged_affirmed).fit(cov_type='cluster',
                                                        cov_kwds={'groups': opinion_YB_judge_YB_dis_merged_affirmed['Circuit']},
                                                        use_t=True)

In [43]:
cluster_court_opinion_YB_judge_YB_dis_merged_affirmed_ols.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:            affirmdummy   R-squared:                       0.002
Model:                            OLS   Adj. R-squared:                  0.002
Method:                 Least Squares   F-statistic:                     26.43
Date:                Thu, 31 Jan 2019   Prob (F-statistic):           0.000245
Time:                        02:59:25   Log-Likelihood:            -2.0817e+05
No. Observations:              288749   AIC:                         4.163e+05
Df Residuals:                  288747   BIC:                         4.164e+05
Df Model:                           1                                         
Covariance Type:              cluster                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.4970      0.017     29.452      0.000       0.460       0.534
dis            0.0944      0.018      5.141      0.000       0.054       0.134
==============================================================================
Omnibus:                     1504.025   Durbin-Watson:                   2.002
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            47803.495
Skew:                          -0.178   Prob(JB):                         0.00
Kurtosis:                       1.039   Cond. No.                         5.74
==============================================================================

Warnings:
[1] Standard Errors are robust tocluster correlation (cluster)
"""

# difference from total

In [44]:
cluster_court_opinion_YB_dis_merged_affirmed_ols = sm.ols(formula='affirmdummy ~ dis', data=opinion_YB_dis_merged_affirmed).fit(cov_type='cluster',
                                                        cov_kwds={'groups': opinion_YB_dis_merged_affirmed['Circuit']},
                                                        use_t=True)

In [45]:
cluster_court_opinion_YB_dis_merged_affirmed_ols.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:            affirmdummy   R-squared:                       0.003
Model:                            OLS   Adj. R-squared:                  0.003
Method:                 Least Squares   F-statistic:                     43.69
Date:                Thu, 31 Jan 2019   Prob (F-statistic):           2.50e-05
Time:                        03:00:03   Log-Likelihood:            -2.0795e+05
No. Observations:              288781   AIC:                         4.159e+05
Df Residuals:                  288779   BIC:                         4.159e+05
Df Model:                           1                                         
Covariance Type:              cluster                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.7051      0.023     30.592      0.000       0.655       0.755
dis           -0.2627      0.040     -6.610      0.000      -0.349      -0.176
==============================================================================
Omnibus:                     1497.456   Durbin-Watson:                   2.002
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            47492.568
Skew:                          -0.178   Prob(JB):                         0.00
Kurtosis:                       1.045   Cond. No.                         12.4
==============================================================================

Warnings:
[1] Standard Errors are robust tocluster correlation (cluster)
"""

# Linear Regression With Clustered Error with Circuit FE to affirmed

In [46]:
cluster_court_opinion_YB_judge_YB_dis_merged_affirmed_ols_fe = sm.ols(formula='affirmdummy ~ dis + C(Circuit)', data=opinion_YB_judge_YB_dis_merged_affirmed).fit(cov_type='cluster',
                                                        cov_kwds={'groups': opinion_YB_judge_YB_dis_merged_affirmed['Circuit']},
                                                        use_t=True)
cluster_court_opinion_YB_judge_YB_dis_merged_affirmed_ols_fe.summary()

/home/jcai/anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:1532: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 13, but rank is 1
  'rank is %d' % (J, J_), ValueWarning)


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:            affirmdummy   R-squared:                       0.016
Model:                            OLS   Adj. R-squared:                  0.016
Method:                 Least Squares   F-statistic:                     11.99
Date:                Thu, 31 Jan 2019   Prob (F-statistic):            0.00469
Time:                        03:00:25   Log-Likelihood:            -2.0613e+05
No. Observations:              288749   AIC:                         4.123e+05
Df Residuals:                  288735   BIC:                         4.124e+05
Df Model:                          13                                         
Covariance Type:              cluster                                         
======================================================================================
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
Intercept              0.4673      0.004    120.499      0.000       0.459       0.476
C(Circuit)[T.1.0]      0.1030      0.001    193.134      0.000       0.102       0.104
C(Circuit)[T.2.0]     -0.0019      0.001     -3.696      0.003      -0.003      -0.001
C(Circuit)[T.3.0]     -0.0249      0.000   -197.547      0.000      -0.025      -0.025
C(Circuit)[T.4.0]      0.0152      0.000    110.606      0.000       0.015       0.015
C(Circuit)[T.5.0]      0.0074      0.001      9.249      0.000       0.006       0.009
C(Circuit)[T.6.0]      0.0360   5.32e-05    676.922      0.000       0.036       0.036
C(Circuit)[T.7.0]      0.1106      0.001    128.088      0.000       0.109       0.112
C(Circuit)[T.8.0]      0.1282      0.001    172.616      0.000       0.127       0.130
C(Circuit)[T.9.0]      0.0337      0.000    224.634      0.000       0.033       0.034
C(Circuit)[T.10.0]     0.1136      0.000    317.700      0.000       0.113       0.114
C(Circuit)[T.11.0]    -0.0115   4.71e-05   -243.388      0.000      -0.012      -0.011
C(Circuit)[T.12.0]    -0.0916      0.000   -571.964      0.000      -0.092      -0.091
dis                    0.0672      0.009      7.865      0.000       0.049       0.086
==============================================================================
Omnibus:                     1443.972   Durbin-Watson:                   2.001
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            45169.905
Skew:                          -0.175   Prob(JB):                         0.00
Kurtosis:                       1.094   Cond. No.                         28.0
==============================================================================

Warnings:
[1] Standard Errors are robust tocluster correlation (cluster)
"""

In [48]:
cluster_court_opinion_YB_dis_merged_affirmed_ols_fe = sm.ols(formula='affirmdummy ~ dis + C(Circuit)', data=opinion_YB_dis_merged_affirmed).fit(cov_type='cluster',
                                                        cov_kwds={'groups': opinion_YB_dis_merged_affirmed['Circuit']},
                                                        use_t=True)
cluster_court_opinion_YB_dis_merged_affirmed_ols_fe.summary()

/home/jcai/anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:1532: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 13, but rank is 1
  'rank is %d' % (J, J_), ValueWarning)


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:            affirmdummy   R-squared:                       0.017
Model:                            OLS   Adj. R-squared:                  0.017
Method:                 Least Squares   F-statistic:                     29.45
Date:                Thu, 31 Jan 2019   Prob (F-statistic):           0.000153
Time:                        03:00:54   Log-Likelihood:            -2.0591e+05
No. Observations:              288781   AIC:                         4.118e+05
Df Residuals:                  288767   BIC:                         4.120e+05
Df Model:                          13                                         
Covariance Type:              cluster                                         
======================================================================================
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
Intercept              0.6587      0.024     27.179      0.000       0.606       0.712
C(Circuit)[T.1.0]      0.0882      0.003     30.626      0.000       0.082       0.094
C(Circuit)[T.2.0]     -0.0180      0.003     -5.946      0.000      -0.025      -0.011
C(Circuit)[T.3.0]     -0.0458      0.003    -13.817      0.000      -0.053      -0.039
C(Circuit)[T.4.0]     -0.0060      0.003     -1.782      0.100      -0.013       0.001
C(Circuit)[T.5.0]     -0.0078      0.003     -2.411      0.033      -0.015      -0.001
C(Circuit)[T.6.0]      0.0131      0.004      3.713      0.003       0.005       0.021
C(Circuit)[T.7.0]      0.0979      0.003     33.330      0.000       0.092       0.104
C(Circuit)[T.8.0]      0.1073      0.004     26.658      0.000       0.099       0.116
C(Circuit)[T.9.0]      0.0143      0.003      4.598      0.001       0.008       0.021
C(Circuit)[T.10.0]     0.0960      0.003     31.248      0.000       0.089       0.103
C(Circuit)[T.11.0]    -0.0240      0.002    -13.131      0.000      -0.028      -0.020
C(Circuit)[T.12.0]    -0.1071      0.002    -49.844      0.000      -0.112      -0.102
dis                   -0.2298      0.035     -6.641      0.000      -0.305      -0.154
==============================================================================
Omnibus:                     1446.936   Durbin-Watson:                   2.001
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            44871.767
Skew:                          -0.175   Prob(JB):                         0.00
Kurtosis:                       1.101   Cond. No.                         29.7
==============================================================================

Warnings:
[1] Standard Errors are robust tocluster correlation (cluster)
"""

# Logistic Regression to affirmed

In [18]:
from statsmodels.discrete.discrete_model import Logit

In [52]:
court_opinion_YB_judge_YB_dis_merged_affirmed_log = Logit.from_formula(formula='affirmdummy ~ dis', data=opinion_YB_judge_YB_dis_merged_affirmed).fit()
court_opinion_YB_judge_YB_dis_merged_affirmed_log.summary()

Optimization terminated successfully.
         Current function value: 0.688310
         Iterations 4


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:            affirmdummy   No. Observations:               288749
Model:                          Logit   Df Residuals:                   288747
Method:                           MLE   Df Model:                            1
Date:                Thu, 31 Jan 2019   Pseudo R-squ.:                0.001262
Time:                        03:03:49   Log-Likelihood:            -1.9875e+05
converged:                       True   LL-Null:                   -1.9900e+05
                                        LLR p-value:                2.927e-111
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -0.0125      0.009     -1.339      0.180      -0.031       0.006
dis            0.3801      0.017     22.402      0.000       0.347       0.413
==============================================================================
"""

In [50]:
court_opinion_YB_dis_merged_affirmed_log = Logit.from_formula(formula='affirmdummy ~ dis', data=opinion_YB_dis_merged_affirmed).fit()
court_opinion_YB_dis_merged_affirmed_log.summary()

Optimization terminated successfully.
         Current function value: 0.687471
         Iterations 4


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:            affirmdummy   No. Observations:               288781
Model:                          Logit   Df Residuals:                   288779
Method:                           MLE   Df Model:                            1
Date:                Thu, 31 Jan 2019   Pseudo R-squ.:                0.002479
Time:                        03:01:56   Log-Likelihood:            -1.9853e+05
converged:                       True   LL-Null:                   -1.9902e+05
                                        LLR p-value:                1.492e-216
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.8296      0.021     39.234      0.000       0.788       0.871
dis           -1.0643      0.034    -31.317      0.000      -1.131      -0.998
==============================================================================
"""

# Linear Regression With Clustered Error to dissentdummy

In [53]:
opinion_YB_dis_merged_dissent = opinion_YB_dis_merged.copy()
opinion_YB_dis_merged_dissent = opinion_YB_dis_merged_dissent[opinion_YB_dis_merged_dissent.dissentdummy.notna()]
opinion_YB_dis_merged_dissent = opinion_YB_dis_merged_dissent[opinion_YB_dis_merged_dissent.dis.notna()]

In [54]:
opinion_YB_judge_YB_dis_merged_dissent = opinion_YB_judge_YB_dis_merged.copy()
opinion_YB_judge_YB_dis_merged_dissent = opinion_YB_judge_YB_dis_merged_dissent[opinion_YB_judge_YB_dis_merged_dissent.dissentdummy.notna()]
opinion_YB_judge_YB_dis_merged_dissent = opinion_YB_judge_YB_dis_merged_dissent[opinion_YB_judge_YB_dis_merged_dissent.dis.notna()]

In [55]:
cluster_court_opinion_YB_judge_YB_dis_merged_dissent_ols = sm.ols(formula='dissentdummy ~ dis', data=opinion_YB_judge_YB_dis_merged_dissent).fit(cov_type='cluster',
                                                        cov_kwds={'groups': opinion_YB_judge_YB_dis_merged_dissent['Circuit']},
                                                        use_t=True)
cluster_court_opinion_YB_judge_YB_dis_merged_dissent_ols.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:           dissentdummy   R-squared:                       0.002
Model:                            OLS   Adj. R-squared:                  0.002
Method:                 Least Squares   F-statistic:                     40.78
Date:                Thu, 31 Jan 2019   Prob (F-statistic):           3.47e-05
Time:                        03:04:07   Log-Likelihood:                -52434.
No. Observations:              288749   AIC:                         1.049e+05
Df Residuals:                  288747   BIC:                         1.049e+05
Df Model:                           1                                         
Covariance Type:              cluster                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.1222      0.010     12.514      0.000       0.101       0.143
dis           -0.0581      0.009     -6.386      0.000      -0.078      -0.038
==============================================================================
Omnibus:                   156742.000   Durbin-Watson:                   2.003
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           785133.824
Skew:                           2.795   Prob(JB):                         0.00
Kurtosis:                       8.833   Cond. No.                         5.74
==============================================================================

Warnings:
[1] Standard Errors are robust tocluster correlation (cluster)
"""

In [56]:
cluster_court_opinion_YB_dis_merged_dissent_ols = sm.ols(formula='dissentdummy ~ dis', data=opinion_YB_dis_merged_dissent).fit(cov_type='cluster',
                                                        cov_kwds={'groups': opinion_YB_dis_merged_dissent['Circuit']},
                                                        use_t=True)
cluster_court_opinion_YB_dis_merged_dissent_ols.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:           dissentdummy   R-squared:                       0.002
Model:                            OLS   Adj. R-squared:                  0.002
Method:                 Least Squares   F-statistic:                     24.28
Date:                Thu, 31 Jan 2019   Prob (F-statistic):           0.000349
Time:                        03:04:10   Log-Likelihood:                -52489.
No. Observations:              288781   AIC:                         1.050e+05
Df Residuals:                  288779   BIC:                         1.050e+05
Df Model:                           1                                         
Covariance Type:              cluster                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.0289      0.011      2.736      0.018       0.006       0.052
dis            0.1049      0.021      4.928      0.000       0.058       0.151
==============================================================================
Omnibus:                   156832.365   Durbin-Watson:                   2.003
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           786039.223
Skew:                           2.796   Prob(JB):                         0.00
Kurtosis:                       8.836   Cond. No.                         12.4
==============================================================================

Warnings:
[1] Standard Errors are robust tocluster correlation (cluster)
"""

# Logit Regression With to dissentdummy

In [25]:
from statsmodels.discrete.discrete_model import Poisson

In [59]:
court_opinion_YB_judge_YB_dis_merged_dissent_log = Logit.from_formula(formula='dissentdummy ~ dis', data=opinion_YB_judge_YB_dis_merged_dissent).fit()
court_opinion_YB_judge_YB_dis_merged_dissent_log.summary()

Optimization terminated successfully.
         Current function value: 0.308485
         Iterations 6


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:           dissentdummy   No. Observations:               288749
Model:                          Logit   Df Residuals:                   288747
Method:                           MLE   Df Model:                            1
Date:                Thu, 31 Jan 2019   Pseudo R-squ.:                0.003046
Time:                        03:06:22   Log-Likelihood:                -89075.
converged:                       True   LL-Null:                       -89347.
                                        LLR p-value:                2.257e-120
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -1.9540      0.015   -132.181      0.000      -1.983      -1.925
dis           -0.6605      0.028    -23.618      0.000      -0.715      -0.606
==============================================================================
"""

In [60]:
court_opinion_YB_dis_merged_dissent_log = Logit.from_formula(formula='dissentdummy ~ dis', data=opinion_YB_dis_merged_dissent).fit()
court_opinion_YB_dis_merged_dissent_log.summary()

Optimization terminated successfully.
         Current function value: 0.308620
         Iterations 6


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:           dissentdummy   No. Observations:               288781
Model:                          Logit   Df Residuals:                   288779
Method:                           MLE   Df Model:                            1
Date:                Thu, 31 Jan 2019   Pseudo R-squ.:                0.002610
Time:                        03:06:54   Log-Likelihood:                -89123.
converged:                       True   LL-Null:                       -89357.
                                        LLR p-value:                1.938e-103
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -3.0591      0.037    -81.882      0.000      -3.132      -2.986
dis            1.2660      0.059     21.448      0.000       1.150       1.382
==============================================================================
"""

# Poisson to dissentdummy

In [61]:
court_opinion_YB_judge_YB_dis_merged_dissent_log_fe = Poisson.from_formula(formula='dissentdummy ~ dis + C(Circuit)', data=opinion_YB_judge_YB_dis_merged_dissent).fit()
court_opinion_YB_judge_YB_dis_merged_dissent_log_fe.summary()

Optimization terminated successfully.
         Current function value: 0.310002
         Iterations 6


<class 'statsmodels.iolib.summary.Summary'>
"""
                          Poisson Regression Results                          
==============================================================================
Dep. Variable:           dissentdummy   No. Observations:               288749
Model:                        Poisson   Df Residuals:                   288735
Method:                           MLE   Df Model:                           13
Date:                Thu, 31 Jan 2019   Pseudo R-squ.:                 0.01239
Time:                        03:07:26   Log-Likelihood:                -89513.
converged:                       True   LL-Null:                       -90636.
                                        LLR p-value:                     0.000
======================================================================================
                         coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------
Intercept             -1.8228      0.036    -51.091      0.000      -1.893      -1.753
C(Circuit)[T.1.0]     -1.0536      0.051    -20.696      0.000      -1.153      -0.954
C(Circuit)[T.2.0]     -0.2533      0.039     -6.558      0.000      -0.329      -0.178
C(Circuit)[T.3.0]     -0.1802      0.041     -4.447      0.000      -0.260      -0.101
C(Circuit)[T.4.0]     -0.0964      0.041     -2.361      0.018      -0.176      -0.016
C(Circuit)[T.5.0]     -0.4950      0.038    -12.877      0.000      -0.570      -0.420
C(Circuit)[T.6.0]     -0.0697      0.039     -1.780      0.075      -0.146       0.007
C(Circuit)[T.7.0]     -0.6131      0.040    -15.361      0.000      -0.691      -0.535
C(Circuit)[T.8.0]     -0.4316      0.039    -11.160      0.000      -0.507      -0.356
C(Circuit)[T.9.0]     -0.1966      0.037     -5.306      0.000      -0.269      -0.124
C(Circuit)[T.10.0]    -0.6690      0.044    -15.320      0.000      -0.755      -0.583
C(Circuit)[T.11.0]    -0.4782      0.049     -9.719      0.000      -0.575      -0.382
C(Circuit)[T.12.0]     0.1375      0.041      3.374      0.001       0.058       0.217
dis                   -0.4518      0.027    -16.752      0.000      -0.505      -0.399
======================================================================================
"""

In [62]:
court_opinion_YB_dis_merged_dissent_log_fe = Poisson.from_formula(formula='dissentdummy ~ dis + C(Circuit)', data=opinion_YB_dis_merged_dissent).fit()
court_opinion_YB_dis_merged_dissent_log_fe.summary()

Optimization terminated successfully.
         Current function value: 0.309846
         Iterations 6


<class 'statsmodels.iolib.summary.Summary'>
"""
                          Poisson Regression Results                          
==============================================================================
Dep. Variable:           dissentdummy   No. Observations:               288781
Model:                        Poisson   Df Residuals:                   288767
Method:                           MLE   Df Model:                           13
Date:                Thu, 31 Jan 2019   Pseudo R-squ.:                 0.01289
Time:                        03:07:56   Log-Likelihood:                -89478.
converged:                       True   LL-Null:                       -90646.
                                        LLR p-value:                     0.000
======================================================================================
                         coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------
Intercept             -2.7859      0.052    -53.147      0.000      -2.889      -2.683
C(Circuit)[T.1.0]     -0.9958      0.051    -19.501      0.000      -1.096      -0.896
C(Circuit)[T.2.0]     -0.1873      0.039     -4.820      0.000      -0.264      -0.111
C(Circuit)[T.3.0]     -0.0857      0.041     -2.097      0.036      -0.166      -0.006
C(Circuit)[T.4.0]     -0.0019      0.041     -0.046      0.964      -0.083       0.079
C(Circuit)[T.5.0]     -0.4392      0.039    -11.357      0.000      -0.515      -0.363
C(Circuit)[T.6.0]      0.0350      0.039      0.886      0.376      -0.042       0.112
C(Circuit)[T.7.0]     -0.5707      0.040    -14.246      0.000      -0.649      -0.492
C(Circuit)[T.8.0]     -0.3492      0.039     -8.933      0.000      -0.426      -0.273
C(Circuit)[T.9.0]     -0.1106      0.037     -2.963      0.003      -0.184      -0.037
C(Circuit)[T.10.0]    -0.5947      0.044    -13.544      0.000      -0.681      -0.509
C(Circuit)[T.11.0]    -0.4202      0.049     -8.529      0.000      -0.517      -0.324
C(Circuit)[T.12.0]     0.2090      0.041      5.115      0.000       0.129       0.289
dis                    1.0843      0.057     19.022      0.000       0.973       1.196
======================================================================================
"""

# Linear Regression With Clustered Error to reverse

In [67]:
opinion_YB_dis_merged_reverse = opinion_YB_dis_merged.copy()
opinion_YB_dis_merged_reverse = opinion_YB_dis_merged_reverse[opinion_YB_dis_merged_reverse.reversedummy.notna()]
opinion_YB_dis_merged_reverse = opinion_YB_dis_merged_reverse[opinion_YB_dis_merged_reverse.dis.notna()]

In [63]:
opinion_YB_judge_YB_dis_merged_reverse = opinion_YB_judge_YB_dis_merged.copy()
opinion_YB_judge_YB_dis_merged_reverse = opinion_YB_judge_YB_dis_merged_reverse[opinion_YB_judge_YB_dis_merged_reverse.reversedummy.notna()]
opinion_YB_judge_YB_dis_merged_reverse = opinion_YB_judge_YB_dis_merged_reverse[opinion_YB_judge_YB_dis_merged_reverse.dis.notna()]

In [65]:
cluster_court_opinion_YB_judge_YB_dis_merged_reverse_ols = sm.ols(formula='reversedummy ~ dis', data=opinion_YB_judge_YB_dis_merged_reverse).fit(cov_type='cluster',
                                                        cov_kwds={'groups': opinion_YB_judge_YB_dis_merged_reverse['Circuit']},
                                                        use_t=True)
cluster_court_opinion_YB_judge_YB_dis_merged_reverse_ols.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:           reversedummy   R-squared:                       0.001
Model:                            OLS   Adj. R-squared:                  0.001
Method:                 Least Squares   F-statistic:                     20.87
Date:                Thu, 31 Jan 2019   Prob (F-statistic):           0.000646
Time:                        03:13:32   Log-Likelihood:            -1.9818e+05
No. Observations:              288749   AIC:                         3.964e+05
Df Residuals:                  288747   BIC:                         3.964e+05
Df Model:                           1                                         
Covariance Type:              cluster                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.3931      0.013     29.150      0.000       0.364       0.423
dis           -0.0601      0.013     -4.568      0.001      -0.089      -0.031
==============================================================================
Omnibus:                    13582.232   Durbin-Watson:                   2.002
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            49257.789
Skew:                           0.570   Prob(JB):                         0.00
Kurtosis:                       1.328   Cond. No.                         5.74
==============================================================================

Warnings:
[1] Standard Errors are robust tocluster correlation (cluster)
"""

In [68]:
cluster_court_opinion_YB_dis_merged_reverse_ols = sm.ols(formula='reversedummy ~ dis', data=opinion_YB_dis_merged_reverse).fit(cov_type='cluster',
                                                        cov_kwds={'groups': opinion_YB_dis_merged_reverse['Circuit']},
                                                        use_t=True)
cluster_court_opinion_YB_dis_merged_reverse_ols.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:           reversedummy   R-squared:                       0.008
Model:                            OLS   Adj. R-squared:                  0.008
Method:                 Least Squares   F-statistic:                     156.4
Date:                Thu, 31 Jan 2019   Prob (F-statistic):           3.04e-08
Time:                        03:14:15   Log-Likelihood:            -1.9720e+05
No. Observations:              288781   AIC:                         3.944e+05
Df Residuals:                  288779   BIC:                         3.944e+05
Df Model:                           1                                         
Covariance Type:              cluster                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.1306      0.016      8.247      0.000       0.096       0.165
dis            0.3801      0.030     12.507      0.000       0.314       0.446
==============================================================================
Omnibus:                    13281.102   Durbin-Watson:                   2.003
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            48018.346
Skew:                           0.562   Prob(JB):                         0.00
Kurtosis:                       1.349   Cond. No.                         12.4
==============================================================================

Warnings:
[1] Standard Errors are robust tocluster correlation (cluster)
"""

# Logit Regression With to reversedummy

In [71]:
court_opinion_YB_judge_YB_dis_merged_reverse_log = Logit.from_formula(formula='reversedummy ~ dis', data=opinion_YB_judge_YB_dis_merged_reverse).fit()
court_opinion_YB_judge_YB_dis_merged_reverse_log.summary()

Optimization terminated successfully.
         Current function value: 0.654695
         Iterations 4


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:           reversedummy   No. Observations:               288749
Model:                          Logit   Df Residuals:                   288747
Method:                           MLE   Df Model:                            1
Date:                Thu, 31 Jan 2019   Pseudo R-squ.:               0.0005761
Time:                        03:19:34   Log-Likelihood:            -1.8904e+05
converged:                       True   LL-Null:                   -1.8915e+05
                                        LLR p-value:                 2.525e-49
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -0.4331      0.010    -45.298      0.000      -0.452      -0.414
dis           -0.2586      0.017    -14.783      0.000      -0.293      -0.224
==============================================================================
"""

In [72]:
court_opinion_YB_dis_merged_reverse_log = Logit.from_formula(formula='reversedummy ~ dis', data=opinion_YB_dis_merged_reverse).fit()
court_opinion_YB_dis_merged_reverse_log.summary()

Optimization terminated successfully.
         Current function value: 0.651210
         Iterations 5


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:           reversedummy   No. Observations:               288781
Model:                          Logit   Df Residuals:                   288779
Method:                           MLE   Df Model:                            1
Date:                Thu, 31 Jan 2019   Pseudo R-squ.:                0.005889
Time:                        03:19:53   Log-Likelihood:            -1.8806e+05
converged:                       True   LL-Null:                   -1.8917e+05
                                        LLR p-value:                     0.000
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -1.5869      0.022    -71.128      0.000      -1.631      -1.543
dis            1.6680      0.036     46.832      0.000       1.598       1.738
==============================================================================
"""